# Travelling salesman problem - Ant Colony Optimization algorithm

**Set of cities:** cities are points with coordinates x, y on a plane with height as z coordinate. 

**Moving:** cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. Two scenarios criteria:
* There are all the direct connections / e.g. 80% of possible connections
* The problem is symmetrical / asymmetrical (if asymmetrical – going up is height +10%, going down: -10%)

**Choosing coordinates:** randomly from the range <-100, 100> for x, y and <0, 50> for z.


In [1]:
import random
import numpy as np

#### **Creating a graph**

In [2]:
#creating a list of cities
random.seed(12345)
cities = []
num_cities = 10
for i in range (0,num_cities):
  cities.append([i, random.uniform(-100,100), random.uniform(-100,100), random.uniform(0,50)])

#function measuring the distance from point 1 to point 2
def euclidean(point1, point2, sym):
  euclidean = ((point1[1]-point2[1])**2 + (point1[2]-point2[2])**2 + (point1[3]-point2[3])**2)**(1/2)
  if sym == True:
    return euclidean
  elif sym == False:
    if point2[2]>point1[2]:
      return euclidean*1.1
    elif point2[2]<point1[2]:
      return euclidean*0.9
    else:
      return euclidean

#function creating the directed graph, connecting nodes with each other with weighted edges (using euclidean function)
def create_graph(nodes, conn, sym):
  poss_conn_num = len(nodes) * (len(nodes) - 1)   #number of all possible connections
  if conn > 1 or conn <= 0: raise ValueError
  edges = []
  for k in range (0, len(nodes)):
    for i in range (0, len(nodes)):
      if i != k:
        edges.append([nodes[k], nodes[i], euclidean(nodes[k], nodes[i], sym)])
  if conn <= 1 and conn > 0: 
    rem_conn_num = round(poss_conn_num*(1-conn))     #number of paths we need to remove
    for i in range (0, rem_conn_num):
      del edges[random.randrange(0, len(edges))]
  return edges

#creating a graph
graph = create_graph(cities, 0.8, False)

**Task:** Use ACO algorithm to approximate the solution of traveling salesman problem.

#### **Ant colony optimisation (ACO)**


In [3]:
def filter_func(edge, curr, visit):
   if edge[0][0] == curr and edge[1][0] not in visit:
       return True
   else:
       return False

def filter_comeback(edge, curr, start):
  if edge[0][0] == curr and edge[1][0] == start:
    return True
  else:
    return False

In [9]:
def ant_colony_optimisation(graph, m, NCMax, alpha, beta, Q, rho):
  #m - number of ants
  #NCMax - maximum number of cycles

  #initialize
  NC = 0
  for path in graph:
    path.extend([1,0])          #[startpoint, endpoint, length, pheromones (r), deltapheromones (delta_r)]
  global_best = (1000000, [])
  ants_tab = np.full((m, num_cities+1), num_cities, dtype=float)   #we fill the array with the value that will never appear in it

  while NC < NCMax:

    print("NC: ", NC)
    #placing m ants randomly on the nodes, setting those nodes as ants' localizations (first tab column)
    ants_tab[:,0] = np.random.choice(num_cities, m, replace=True)
    #lengths of the m ants' tours
    ants_len = np.zeros(shape=(m,))

    #finding the whole path for each ant
    for s in range(1, num_cities):          #we skip the comeback city
      for k in range(0, m):

        #if the current city is actually the city
        if not np.isnan(ants_tab[k, s-1]):      #optimization
          #possible paths to go
          graph_filtered = list(filter(lambda seq: filter_func(seq, ants_tab[k,s-1], ants_tab[k,:]), graph))
        else:
          graph_filtered = []

        #the current city is not a city or it is, but we have no way out
        if len(graph_filtered) == 0:      #optimization
          ants_tab[k, s] = None
          ants_len[k] = None
        else:
          #probability value assignment
          sum_influence = sum([c[3]**alpha * (1/c[2])**beta for c in graph_filtered])
          probabilities = []
          for c in graph_filtered:
            city_influence = c[3]**alpha * (1/c[2])**beta
            p = city_influence/sum_influence
            probabilities.append(p)
          #we draw one city based on probabilities and make it the next one
          goto = random.choices(graph_filtered, weights=tuple(probabilities), k=1)[0]
          ants_tab[k, s] = goto[1][0]
          ants_len[k] = ants_len[k] + goto[2]

    #comeback - move the ants to the startpoint city (if it's possible)
    s = num_cities
    for k in range(0, m):
      if np.isnan(ants_tab[k, s-1]):      #np.isnan crucial! we cannot use is None even though we added it
        ants_tab[k, s] = None
      else:
        comeback_search = list(filter(lambda seq: filter_comeback(seq, ants_tab[k,s-1], ants_tab[k,0]), graph))
        if len(comeback_search) == 0:
          ants_tab[k, s] = None
          ants_len[k] = None
        else:
          comeback_search = comeback_search[0]
          ants_tab[k, s] = comeback_search[1][0]
          ants_len[k] = ants_len[k] + comeback_search[2]

    #the shortest tour found
    print("ants_tab: ", ants_tab)
    best = (ants_len[np.nanargmin(ants_len)], ants_tab[np.nanargmin(ants_len), :])
    if best[0] < global_best[0]:
      global_best = best
    print("global_best: ", global_best)
    
    #add pheromones on the generally visited edges (updating delta_r)
    for path in graph:
      for k in range(0, m):
        #check if an ant went that way (just for the ants that finished the journey!)
        if not np.isnan(ants_len[k]):
          ind = np.where(ants_tab[k, :num_cities] == path[0][0])
          if len(ind) != 0 and ants_tab[k, ind[0]+1] == path[1][0]:
            delta_r_k = Q/ants_len[k]
          else:
            delta_r_k = 0
          path[4] = path[4] + delta_r_k
    
    #pheromones level on the edges (updating r) - evaporation and adding delta_r
    for path in graph:
      path[3] = rho*path[3] + path[4]       #[startpoint, endpoint, length, pheromones (r), deltapheromones (delta_r)]

    NC = NC + 1
    for path in graph: path[4] = 0
    ants_tab = np.full((m, num_cities+1), num_cities, dtype=float)

  return global_best

In [11]:
g = create_graph(cities, 0.8, False)
ant_colony_optimisation(g, 20, 30, 0.5, 0.5, 3, 0.7)

NC:  0
ants_tab:  [[ 4.  0.  8.  3.  1.  9.  2.  6.  5.  7.  4.]
 [ 3.  0.  4.  7.  9.  2.  1.  8.  6.  5. nan]
 [ 0.  8.  4.  1.  7.  9.  3.  5.  6.  2.  0.]
 [ 7.  4.  1.  5.  8.  0.  6.  2.  9.  3. nan]
 [ 4.  1.  8.  6.  3.  0.  9.  5.  2.  7.  4.]
 [ 4.  3.  1.  9.  2.  6.  7.  5.  0.  8.  4.]
 [ 5.  8.  4.  0.  6.  2.  1.  3.  9.  7.  5.]
 [ 2.  8.  4.  3.  0.  5.  9.  6.  1.  7.  2.]
 [ 1.  3.  4.  7.  5.  6.  0.  9.  2.  8. nan]
 [ 1.  7.  4.  0.  9.  6.  5.  8.  2. nan nan]
 [ 8.  2.  7.  6.  3.  9.  0.  4.  1.  5.  8.]
 [ 1.  2.  6.  3.  4.  0.  9.  7.  8. nan nan]
 [ 1.  4.  2.  6.  0.  5.  8.  3.  9.  7. nan]
 [ 4.  3.  9.  2.  6.  5.  8.  7.  0.  1.  4.]
 [ 5.  7.  2.  8.  6.  3.  1.  4.  0.  9.  5.]
 [ 5.  8.  2.  4.  3.  1.  9.  6.  7.  0.  5.]
 [ 2.  4.  1.  9.  5.  8.  0.  3. nan nan nan]
 [ 0.  1.  9.  2.  6.  3.  5.  7.  8.  4.  0.]
 [ 7.  0.  4.  1.  2.  6.  3.  8.  9.  5.  7.]
 [ 4.  1.  2.  6.  0.  9.  5.  7.  3.  8.  4.]]
global_best:  (848.6960918147795, array([

(680.9758092355937, array([8., 4., 0., 9., 6., 2., 1., 7., 3., 5., 8.]))

The result improves - it becomes more visible when we increase the rho factor (pheromones don't evaporate so easily). 

However, finally ants don't choose exactly the same roads, because:
1. their next city choice is a product of some random choice with probabilities - the road with the most pheromones is the most probable to be chosen, however it is not sure,
2. they start from various startpoints, so it is not easy to compare.

Anyway, we can tell that indeed some paths (A -> B) are more frequently observed during ants' travels.